In [1]:
import requests
import re
from time import sleep
from typing import List, Tuple, Union
from datetime import datetime, timezone
from dateutil.parser import isoparse
from dataclasses import dataclass, asdict
from dacite import from_dict, Config

from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector.errors import IntegrityError
from tqdm.auto import tqdm

from IPython.core.debugger import set_trace

# Data Collection

## MAL API connection class

In [37]:
class MAL_API:

    USERS_ANIMELIST_URL = 'https://api.myanimelist.net/v2/users/{}/animelist?fields=list_status&limit=100'
    ANIME_DETAILS_URL = (
        'https://api.myanimelist.net/v2/anime/{}?fields=id,title,start_date,end_date,mean,rank,popularity,'
        'num_list_users,num_scoring_users,nsfw,media_type,status,genres,num_episodes,start_season,broadcast,'
        'source,average_episode_duration,rating,related_anime,recommendations,studios,statistics'
    )
    # Minimum time delay between two subsequent requests.
    API_CALL_TIME_LIMIT = 0.3
    # Lists with keys from animelist dictionaries of variables to be extracted.
    list_entry_node_keys = ['id', 'title']
    list_entry_list_status_keys = ['status', 'score', 'num_episodes_watched', 'updated_at', 'start_date', 'finishi_date']

    def __init__(self, client_id: str) -> None:
        """Constructor that returns a MAL_API object with CLIENT_ID defined in headers
           and convenience functions for making calls to the API.

        Args:
            client_id (str): Authorization token.
        """
        self.CLIENT_ID = client_id
        self.api_headers = {
            'X-MAL-CLIENT-ID' : self.CLIENT_ID
        }
        self.last_api_request_time = datetime.now()

    def api_call(self, request_url: str, request_headers: dict) -> dict:
        """
            The only method from which api calls are made directly.
            Ensures that time difference between subsequent requests is at least that specified in `API_CALL_TIME_LIMIT`.

        Args:
            request_url (str): API call url.
            request_headers (dict): API request headers.

        Returns:
            dict: API response.
        """
        time_now = datetime.now()
        last_request_time_diff = (time_now - self.last_api_request_time).total_seconds()
        # If time since last request is less than limit, sleep for the duration of time difference.
        if last_request_time_diff < self.API_CALL_TIME_LIMIT:
            sleep(self.API_CALL_TIME_LIMIT - last_request_time_diff)
        response_json = requests.get(request_url, headers=request_headers).json()
        self.last_api_request_time = datetime.now()
        return response_json
    
    def __parse_list_entry(self, entry: dict) -> dict:
        """Parse an entry of animelist. Select important variables and merge dictionaries.

        Args:
            entry (dict): Anime entry in a animelist from the API.

        Returns:
            dict: Parsed entry of animelist as a single dictionary.
        """
        node = entry['node']
        node_parsed = {key: node.get(key) for key in self.list_entry_node_keys}
        list_status = entry['list_status']
        list_status_parsed = {key: list_status.get(key) for key in self.list_entry_list_status_keys}
        # Not all entries have start and/or finished dates. Users have not started or finished some of the anime on their lists.
        # If they don't exist add them with `None` values.
        # if 'start_date' in list_status.keys():
        #     list_status_parsed['start_date'] = list_status['start_date']
        # else:
        #     list_status_parsed['start_date'] = None
        # if 'finish_date' in list_status.keys():
        #     list_status_parsed['finish_date'] = list_status['finish_date']
        # else:
        #     list_status_parsed['finish_date'] = None
        parsed_animelist = {**node_parsed, **list_status_parsed}
        return parsed_animelist

    def get_users_animelist(self, username: str) -> list:
        """Gets an animelist of a user by theirs username from the MAL API.

        Args:
            username (str): Username of the user, whose animelist will be retrieved.

        Returns:
            list: Each element of the list is a single anime entry in the animelist.
                  A single anime entry is a dictionary with selected values of the entry.
        """
        # End condition response['paging'] - no 'next' key in dictionary.
        animelist_url = self.USERS_ANIMELIST_URL.format(username)
        animelist = []
        end_of_list = False
        while not end_of_list:
            animelist_page = self.api_call(request_url=animelist_url, request_headers=self.api_headers)
            # Check special cases.
            if 'error' in animelist_page.keys():
                if animelist_page['error'] == 'not_permitted':
                    end_of_list = True
                elif animelist_page['error'] == 'not_found':
                    end_of_list = True
            else:
                # Add page to animelist list.
                animelist.extend([self.__parse_list_entry(anime_entry) for anime_entry in animelist_page['data']])
                # Change url to next page. Checking for end condition.
                paging = animelist_page['paging']
                if 'next' in paging.keys():
                    animelist_url = paging['next']
                else:
                    end_of_list = True
        return animelist
    
    def get_anime_details(self, anime_id: int) -> dict:
        """Makes a call to the API and retrieves details about an anime by its ID.

        Args:
            anime_id (int): ID of the anime in MAL DB to be retrieved.

        Returns:
            dict: Details of an anime.
        """
        return self.api_call(request_url=self.ANIME_DETAILS_URL.format(anime_id), request_headers=self.api_headers)
        

CLIENT_ID = '32df078edb7cabcd2eb77f026384e66b'
mal_api = MAL_API(client_id=CLIENT_ID)

## DB connection class

In [38]:
@dataclass
class DataClassDatetime:
    """Parent class for other dataclasses that have a datetime field.
        Used for a single datetime parsing function.
    """
    def validate_and_parse_date(self, date: str) -> Union[None, str]:
            """Tries to convert date to iso date format and returns it as UTC datetime. 
                If `date` is `None` or cannot be converted to iso format return `None`.

            Args:
                date (str): Date to be validated and parsed.

            Returns:
                Union[None, str]: `date` argument if it matches the expected format and `None` otherwise or when `None` is passed.
            """
            if date is None:
                return None
            else:
                try:
                    date = isoparse(date).astimezone(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
                except ValueError:
                    # If time format is not as specified return None to be inserted into db.
                    return None
            return date

@dataclass
class Anime(DataClassDatetime):
    id: int
    title: str
    start_date: str = None
    end_date: str = None
    mean_score: float = None
    anime_rank: int = None
    popularity: int = None
    num_list_users: int = None
    num_scoring_users: int = None
    nsfw: str = None
    media_type: str = None
    status: str = None
    num_episodes: int = None
    season_year: int = None
    season: str = None
    broadcast_day_of_week: str = None
    broadcast_start_time: str = None
    source: str = None
    average_episode_duration: int = None
    rating: str = None
    status_watching: int = None
    status_completed: int = None
    status_on_hold: int = None
    status_dropped: int = None
    status_plan_to_watch: int = None

    def __post_init__(self):
        self.start_date = self.validate_and_parse_date(self.start_date)
        self.end_date = self.validate_and_parse_date(self.end_date)

@dataclass
class ListEntry(DataClassDatetime):
    anime_id: int
    user_id: int
    status: str = None
    score: float = None
    num_episodes_watched: int = None
    updated_at: str = None
    start_date: str = None
    finish_date: str = None

    def __post_init__(self):
        self.updated_at = self.validate_and_parse_date(self.updated_at)
        self.start_date = self.validate_and_parse_date(self.start_date)
        self.finish_date = self.validate_and_parse_date(self.finish_date)


@dataclass
class Studio:
    id: int
    name: str = None


@dataclass
class AnimeStudio:
    anime_id: int
    studio_id: int


@dataclass
class Genre:
    id: int
    name: str = None


@dataclass
class AnimeGenre:
    anime_id: int
    genre_id: int


@dataclass
class RelatedAnime:
    anime_id: int
    related_anime_id: int
    relation_prequel: int = None

    def __post_init__(self):
        # This comes from api as a string. Needs to be transformed.
        self.relation_prequel = 1 if self.relation_prequel == 'prequel' else 0


@dataclass
class MALAnimeRecommendation:
    anime_id: int
    recommended_anime_id: int
    num_recommendations: int = None


class AnimeDatabase:
    """First populate the User table. Then populate the List_Entry table.
        If an anime is not present in the db, when adding an entry to List_Entry table first add anime to db.

        Commit after each list entry.
    """

    USERS_PAGE_URL = 'https://myanimelist.net/users.php'
    # ANIME_TABLE_FIELDS = [
    #     'id', 'title', 'start_date', 'end_date', 'mean_score', 'anime_rank',
    #     'popularity', 'num_list_users', 'num_scoring_users', 'nsfw',
    #     'media_type', 'status', 'num_episodes', 'season_year', 'season',
    #     'broadcast_day_of_week', 'broadcast_start_time', 'source',
    #     'average_episode_duration', 'rating', 'status_watching',
    #     'status_completed', 'status_on_hold', 'status_dropped',
    #     'status_plan_to_watch'
    # ]

    def __init__(self, host: str, user: str, password: str, database: str, mal_api: MAL_API, anime_class: type) -> None:
        """Construct an object with a database connection object and cursor for querying the db.

        Args:
            host (str): Database host.
            user (str): Database username.
            password (str): Database user password.
            database (str): Database name.
            mal_api (MAL_API): An object of MAL_API class. Used for retrieving anime details when inserting
                                an entry of users animelist and when an anime is not yet present in db.
            anime_class (type): Class object of the dataclass that maps to Anime table in db. Used for extracting table fields.
        """
        self.mal_api = mal_api
        self.db_conn = mysql.connector.connect(host=host,
                                                  user=user,
                                                  password=password,
                                                  database=database)
        self.cursor = self.db_conn.cursor(buffered=True)
        self.ANIME_TABLE_FIELDS = list(anime_class.__dict__['__annotations__'].keys())

    def execute_queries(self, queries: List[str], queries_params: List[dict]) -> None:
        """Convenience function for running one or more queries. The queries are committed once all have passed.

        Args:
            queries (List[str]): List of queries to execute.
            queries_params (List[dict]): List of queries params to execute together with queries.
                                         Must be the same length as queries.

        Raises:
            ValueError: Error is raised if the lengths of two passed lists do not match.
        """
        if len(queries) != len(queries_params):
            raise ValueError('The lengths of `queries` and `queries_params` do not match.')
        for query, query_params in zip(queries, queries_params):
            if query_params:
                self.cursor.execute(query, query_params)
            else:
                self.cursor.execute(query)
        self.db_conn.commit()
    
    def insert_user(self, username: str) -> None:
        """Insert user to User table in DB.

        Args:
            username (str): Username of the user in MAL DB.
        """
        query = 'INSERT INTO User (username, list_collected) VALUES (%(username)s, %(list_collected)s);'
        self.execute_queries(queries=[query], queries_params=[{'username': username, 'list_collected': 0}])
    
    def insert_users_animelist_entry(self, user_id: int, list_entry: dict) -> None:
        """Inserts an entry of users animelist to `List_Entry` table.

        Args:
            user_id (int): ID of user in the database.
            list_entry (dict): List entry data in the format returned by MAL API: https://myanimelist.net/apiconfig/references/api/v2#operation/users_user_id_animelist_get.
        """
        query_values = list_entry
        # title not needed for this table.
        query_values.pop('title')
        # Change anime_id key to comply with table filed name.
        query_values['anime_id'] = query_values.pop('id')
        query_values['user_id'] = user_id
        # There is a typo in the finish_date field returned by the API as of writing this code.
        if 'finish_date' not in query_values.keys():
            if 'finishi_date' in query_values.keys():
                query_values['finish_date'] = query_values.pop('finishi_date')
        # Create a dataclass object to ensure fields are ready for db insert.
        list_entry_dc = from_dict(data_class=ListEntry, data=query_values, config=Config(check_types=False))

        # Check if anime already in db, if not, first add it.
        self.cursor.execute(f'SELECT * FROM Anime WHERE id = {list_entry_dc.anime_id};')
        result = self.cursor.fetchone()
        if not result:
            self.insert_anime(anime_details=self.mal_api.get_anime_details(list_entry_dc.anime_id))

        query = (
            'INSERT INTO List_Entry (anime_id, user_id, status, score,'
            'num_episodes_watched, updated_at, start_date, finish_date) '
            'VALUES (%(anime_id)s, %(user_id)s, %(status)s, %(score)s,'
            '%(num_episodes_watched)s, %(updated_at)s, %(start_date)s, %(finish_date)s);'
        )
        self.execute_queries(queries=[query], queries_params=[asdict(list_entry_dc)])

    def get_queries_insert_studios(self, studios: List[dict]) -> Tuple[List[str], dict]:
        """Checks if studios are already in the db. If not returns an insert query for each studio not present in the db.

        Args:
            studios (List[dict]): List of dictionaries with studios, with keys `id` and `name`.

        Returns:
            Tuple[List[str], dict]: Insert queries and query parameters for each query.
        """
        queries = []
        queries_params = []
        for studio in studios:
            # Create a dataclass object to ensure fields are ready for db insert.
            studio_dc = from_dict(data_class=Studio, data=studio, config=Config(check_types=False))
            # Check if studio exists.
            self.cursor.execute(f'SELECT * FROM Studio WHERE id = {studio_dc.id};')
            result = self.cursor.fetchone()
            if not result:
                # If not add insert query and studios parameters.
                queries.append('INSERT INTO Studio (id, name) VALUES (%(id)s, %(name)s);')
                queries_params.append(asdict(studio_dc))
        return queries, queries_params

    def get_queries_insert_anime_studios(self, anime_id: int, studios: List[dict]) -> Tuple[List[str], dict]:
        """Assuming that anime and all studios from arguments are already present in the db.
           For each studio add query together with parameters that will insert the relationships between Anime and Studio
           to Anime_Studio table.

        Args:
            anime_id (int): id of the Anime.
            studios (List[dict]): List of studios that are related to the Anime. Each dictionary element in the format of MAL API.

        Returns:
            Tuple[List[str], dict]: Insert queries and query parameters for each query.
        """
        queries = []
        queries_params = []
        for studio in studios:
            # Create a dataclass object to ensure fields are ready for db insert.
            anime_studio = from_dict(data_class=AnimeStudio,
                                     data={'anime_id': anime_id, 'studio_id': studio['id']},
                                     config=Config(check_types=False))
            queries.append('INSERT INTO Anime_Studio (anime_id, studio_id) VALUES (%(anime_id)s, %(studio_id)s);')
            # Create a dictionary with keys compliant with db table.
            queries_params.append(asdict(anime_studio))
        return queries, queries_params
    
    def get_queries_insert_genres(self, genres: List[dict]) -> Tuple[List[str], dict]:
        """Checks if genres are already in the db. If not returns an insert query for each genre not present in the db.

        Args:
            genres (List[dict]): List of dictionaries with genres, with keys `id` and `name`.

        Returns:
            Tuple[List[str], dict]: Insert queries and query parameters for each query.
        """
        queries = []
        queries_params = []
        for genre in genres:
            # Create a dataclass object to ensure fields are ready for db insert.
            genre = from_dict(data_class=Genre, data={'id': genre['id'], 'name': genre['name']},
                              config=Config(check_types=False))
            # Check if genre exists.
            self.cursor.execute(f'SELECT * FROM Genre WHERE id = {genre.id};')
            result = self.cursor.fetchone()
            if not result:
                # If not add insert query and genre parameters.
                queries.append('INSERT INTO Genre (id, name) VALUES (%(id)s, %(name)s);')
                queries_params.append(asdict(genre))
        return queries, queries_params

    def get_queries_insert_anime_genres(self, anime_id: int, genres: List[dict]) -> Tuple[List[str], dict]:
        """Assuming that anime and all genres from arguments are already present in the db.
           For each genre add query together with parameters that will insert the relationships between Anime and Genre
           to Anime_Genre table.

        Args:
            anime_id (int): id of the Anime.
            genres (List[dict]): List of genres of the Anime. Each dictionary element in the format of MAL API.

        Returns:
            Tuple[List[str], dict]: Insert queries and query parameters for each query.
        """
        queries = []
        queries_params = []
        for genre in genres:
            # Create a dataclass object to ensure fields are ready for db insert.
            anime_genre = from_dict(data_class=AnimeGenre, data={'anime_id': anime_id, 'genre_id': genre['id']},
                                    config=Config(check_types=False))
            queries.append('INSERT INTO Anime_Genre (anime_id, genre_id) VALUES (%(anime_id)s, %(genre_id)s);')
            # Create a dictionary with keys compliant with db table.
            queries_params.append(asdict(anime_genre))
        return queries, queries_params

    def get_queries_insert_related_anime(self, anime_id: int, related_anime: List[dict]) -> Tuple[List[str], dict]:
        """Generates and returns queries for inserting anime related to anime supplied in `anime_id`.

        Args:
            anime_id (int): id of the anime other anime are related to.
            related_anime (List[dict]): Properties of the related anime. Format as in MAL API.

        Returns:
            Tuple[List[str], dict]: Insert queries and query parameters for each query.
        """
        queries = []
        queries_params = []
        for ra in related_anime:
            # Create a dataclass object to ensure fields are ready for db insert.
            ra_dict = {
                'anime_id': anime_id,
                'related_anime_id': ra['node']['id'],
                'relation_prequel': ra['relation_type']
            }
            related_anime_dc = from_dict(data_class=RelatedAnime, data=ra_dict, config=Config(check_types=False))
            queries.append(
                'INSERT INTO Related_Anime (anime_id, related_anime_id, relation_prequel) '
                'VALUES (%(anime_id)s, %(related_anime_id)s, %(relation_prequel)s);'
            )
            queries_params.append(asdict(related_anime_dc))
        return queries, queries_params
    
    def get_queries_insert_mal_recommendations(self, anime_id: int, recommendations: List[dict]) -> Tuple[List[str], dict]:
        """Generates insert queries for inserting anime recommended to anime specified by `anime_id` to MAL_Anime_Recommendation table.

        Args:
            anime_id (int): id of anime other anime are recommended to.
            recommendations (List[dict]): Properties of recommended anime. Format as in MAL API.

        Returns:
            Tuple[List[str], dict]: Insert queries and query parameters for each query.
        """
        queries = []
        queries_params = []
        for recommendation in recommendations:
            # Create a dataclass object to ensure fields are ready for db insert.
            recommendation_dict = {
                'anime_id': anime_id,
                'recommended_anime_id': recommendation['node']['id'],
                'num_recommendations': recommendation['num_recommendations']
            }
            recommendation_dc = from_dict(data_class=MALAnimeRecommendation, data=recommendation_dict, config=Config(check_types=False))
            queries.append(
                'INSERT INTO MAL_Anime_Recommendation (anime_id, recommended_anime_id, num_recommendations) '
                'VALUES (%(anime_id)s, %(recommended_anime_id)s, %(num_recommendations)s);'
            )
            queries_params.append(asdict(recommendation_dc))
        return queries, queries_params

    def insert_anime(self, anime_details: dict) -> None:
        """First generate queries as strings (together with queries params) in an appropriate order to ensure
           no foreign key constraints issues. Second pass those to the `execute_queries` method.

        Args:
            anime_details (dict): Details of an anime as in MAL API.
        """
        queries = []
        queries_params = []
        
        # season is unknown for some anime.
        if 'start_season' in anime_details.keys():
            season = anime_details.pop('start_season')
        else:
            season = {
                'year': None,
                'season': None
            }
        anime_details['season_year'] = season['year']
        anime_details['season'] = season['season']

        # broadcast is unknown for some anime.
        if 'broadcast' in anime_details.keys():
            broadcast = anime_details.pop('broadcast')
        else:
            broadcast = {
                'day_of_the_week': None,
                'start_time': None
            }
        anime_details['broadcast_day_of_week'] = broadcast.get('day_of_the_week')
        anime_details['broadcast_start_time'] = broadcast.get('start_time')
        
        lists_statuses = anime_details.pop('statistics')
        anime_details.update({'status_' + key: val for key, val in lists_statuses['status'].items()})
        anime_fields = {key: val for key, val in anime_details.items() if key in self.ANIME_TABLE_FIELDS}
        anime_id = anime_fields['id']
        
        # Create a dataclass to ensure all fields are correct for db insert.
        anime = from_dict(data_class=Anime, data=anime_details, config=Config(check_types=False))
        # Insert Anime queries.
        anime_query = 'INSERT INTO Anime (' + ', '.join(self.ANIME_TABLE_FIELDS) + ') VALUES (%(' + ')s, %('.join(self.ANIME_TABLE_FIELDS) + ')s);'
        queries.append(anime_query)
        queries_params.append(asdict(anime))


        studios = anime_details.get('studios', [])
        # Insert studios queries, if studio not in db.
        studio_queries, studio_queries_params = self.get_queries_insert_studios(studios=studios)
        # Insert anime_studio relationships.
        anime_studio_queries, anime_studio_queries_params = self.get_queries_insert_anime_studios(anime_id=anime_id,
                                                                                                  studios=studios)
        genres = anime_details.get('genres', [])
        # Insert genres queries, if genre not in db.
        genre_queries, qenre_queries_params = self.get_queries_insert_genres(genres=genres)
        # Insert anime_genre relationships.
        anime_genre_queries, anime_genre_queries_params = self.get_queries_insert_anime_genres(anime_id=anime_id,
                                                                                               genres=genres)
        # Insert related_anime queries.
        related_anime_queries, related_anime_queries_params = self.get_queries_insert_related_anime(anime_id=anime_id,
                                                                                                    related_anime=anime_details['related_anime'])
        # Insert MAL_anime_recommendation queries.
        recommended_anime_queries, recommended_anime_queries_params = self.get_queries_insert_mal_recommendations(anime_id=anime_id,
                                                                                                                 recommendations=anime_details['recommendations'])
        # Add all queries in the appropriate order of execution to lists.
        queries.extend(studio_queries)
        queries.extend(anime_studio_queries)
        queries.extend(genre_queries)
        queries.extend(anime_genre_queries)
        queries.extend(related_anime_queries)
        queries.extend(recommended_anime_queries)
        queries_params.extend(studio_queries_params)
        queries_params.extend(anime_studio_queries_params)
        queries_params.extend(qenre_queries_params)
        queries_params.extend(anime_genre_queries_params)
        queries_params.extend(related_anime_queries_params)
        queries_params.extend(recommended_anime_queries_params)
        # Finally execute the queries.
        self.execute_queries(queries=queries, queries_params=queries_params)
        
    def populate_users_animelist(self,user_id: int, username: str) -> None:
        """Populate animelist of `username` by adding entries to List_Entry table.

        Args:
            user_id (int): ID of the user in the db.
            username (str): Name of the users animelist of whose is to be populated in the db. 
        """
        users_anime_list = self.mal_api.get_users_animelist(username=username)
        for entry in tqdm(users_anime_list, desc='Populating users animelist', leave=False):
            try:
                self.insert_users_animelist_entry(user_id=user_id, list_entry=entry)
            except IntegrityError:
                # If entry is already in the db just pass.
                # Makes it easier to rerun the same user in the event of a failure.
                pass

    def populate_users(self, n_pages: int) -> None:
        """Retrieves username from MAL users.php page. Each page contains 20 users and the number
           of pages to be retrieved is given as the argument `n_pages`.

           After the retrieval of usernames from all pages, execute a query that deletes duplicates.

        Args:
            n_pages (int): Number of get request to users.php page to retrieve.
        """
        # with tqdm(range(n_pages), desc='Populating Users table') as pbar:
        for i in tqdm(range(n_pages), desc='Populating Users table'):
            if i % 100 == 0:
                # TODO uncomment
                sleep(1)#20)
            else:
                sleep(2)
            users_soup = BeautifulSoup(requests.get(self.USERS_PAGE_URL).text, 'html.parser')
            user_names = [user_soup.find('a').get_text() for user_soup in users_soup.find('td').find_all('td')]
            for username in user_names:
                self.insert_user(username)
        remove_duplicate_users_query = (
            'DELETE ud '
                'FROM User ud '
                    'INNER JOIN (SELECT '
                                    'u1.id, u1.username, u2.CNT, ROW_NUMBER() OVER(PARTITION BY u1.username ORDER BY u1.username, u1.id) AS RowRank '
                                    'FROM User AS u1 '
                                        'INNER JOIN (SELECT '
                                                        'username, COUNT(*) as CNT '
                                                        'FROM User '
                                                        'GROUP BY username '
                                                        'HAVING COUNT(*) > 1'
                                                    ') AS u2 ON u1.username=u2.username'
                                ') AS u3 ON ud.id=u3.id '
                    'WHERE u3.RowRank!=1;'
        )
        self.execute_queries([remove_duplicate_users_query], [dict()])
    
    def populate_animelists(self) -> None:
        """
            Selects users from db, that have not had their anime list collected yet.
            Iterates through them and calls `AnimeDatabase.populate_users_animelist` for each of them.
        """
        # Separate buffered cursor to allow for queries inside function calls inside the loop.
        # This way the `cursor_buffered` maintains the query response for iterating through SELECT Users.
        cursor_buffered = self.db_conn.cursor(buffered=True)
        cursor_buffered.execute('SELECT COUNT(*) FROM User WHERE list_collected=0;')
        users_count = cursor_buffered.fetchone()[0]
        cursor_buffered.execute('SELECT id, username FROM User WHERE list_collected=0;')
        for user_id, username in tqdm(cursor_buffered, total=users_count, desc='Collecting users animelists', position=0):
            self.populate_users_animelist(user_id=user_id, username=username)
            # Update user to indicate that their list has been collected.
            self.execute_queries(queries=['UPDATE User SET list_collected=1 WHERE id=%(id)s;'],
                                 queries_params=[{'id': user_id}])
        cursor_buffered.close()
    
    def cleanup(self) -> None:
        self.cursor.close()
        self.db_conn.close()


## Scraping usernames

I was considering supplying usernames with all members from top 100 anime. However a list of members of an anime only contains 7500 users, that have recently updated a this anime on their list. Therefore I will only use the random sample from the `https://myanimelist.net/users.php` page of recently active users.

In [40]:
ani_db.cleanup()
ani_db = AnimeDatabase(host='localhost', user='anidb', password='anidb', database='Anime_tet', mal_api=mal_api, anime_class=Anime)

In [130]:
ani_db.populate_users(n_pages=5)

Populating Users table:   0%|          | 0/5 [00:00<?, ?it/s]

In [41]:
ani_db.cursor.execute('SELECT COUNT(*) FROM User;')
number_collected_users = ani_db.cursor.fetchone()[0]
print(f'{number_collected_users} usernames were collected')

100 usernames were collected


## Collecting users anime lists

Collecting anime lists of all users in the db.

In [42]:
ani_db.populate_animelists()

Populating users animelist:   0%|          | 0/426 [00:00<?, ?it/s]

Populating users animelist:   0%|          | 0/149 [00:00<?, ?it/s]

Populating users animelist:   0%|          | 0/339 [00:00<?, ?it/s]

Populating users animelist: 0it [00:00, ?it/s]

Populating users animelist:   0%|          | 0/297 [00:00<?, ?it/s]

Populating users animelist:   0%|          | 0/285 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [36]:
ani_db.cleanup()